<a href="https://colab.research.google.com/github/andygma567/LLM-experiments/blob/main/OpenAlpaca_3BT_test_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Something went wrong when trying to use the AutoModel and AutoTokenizer stuff... I got it working again by using the Llama specific stuff

In [ ]:
!nvidia-smi

Sun Jun 18 14:37:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip -q install bitsandbytes accelerate # required for efficient loading / inference
# !pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q transformers
!pip install -q gradio
!pip install -q SentencePiece # required by Llama
!pip install -q xformers # required to use pipelines with OpenAlpaca

!pip -q install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.4 MB/s eta 0:00:0

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer
# from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GenerationConfig, pipeline

import torch

# import gradio as gr

In [ ]:
# LangChain stuff
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

In [ ]:
import warnings
# warnings.filterwarnings("ignore")

# Load Model

All of the templates are set up for OpenAlpaca and will need to be adjusted if I want to use the chavinlo/Alpaca. For example, I think 'Response' needs to be changed to 'Answer'

In [ ]:
drive_checkpoint = '/content/drive/MyDrive/openalpaca_7b_700bt_preview'
HF_checkpoint = "openllmplayground/openalpaca_7b_700bt_preview"

In [ ]:
%%time
tokenizer = LlamaTokenizer.from_pretrained(
    HF_checkpoint,
    use_fast=True,
    )

CPU times: user 104 ms, sys: 15 ms, total: 119 ms
Wall time: 860 ms


In [ ]:
%%time
base_model = LlamaForCausalLM.from_pretrained(
    drive_checkpoint,
    device_map="auto",
    offload_folder="offload",
    # offload_state_dict = True, # I only need to use this if I'm really hurting for GPU memory
    load_in_8bit=True,
    torch_dtype=torch.float16, # This is needed for bits and bytes
    )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

CPU times: user 13.3 s, sys: 34.2 s, total: 47.5 s
Wall time: 5min 59s


# Test generating text with a method

In [ ]:
# I borrowed most of these settings from a youtube demo
# I think most of these settings are unnecessary
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.2,
    max_new_tokens=1024, # According to the model card this was how it was trained with 1024
    use_cache=True,
    do_sample=True,
)

In [ ]:
def use_OpenAlpaca(input: str) -> str:
  prompt_no_input = f'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{input}\n\n### Response:'
  # encode returns just the input tokens, no maskes etc.
  tokens = tokenizer.encode(prompt_no_input) # tensors are being returned as python vars by default (I think)
  tokens = torch.LongTensor(tokens).unsqueeze(0).cuda()

  length = len(tokens[0])
  # Take this out for testing with the LangChain LLM wrapper
  # print("Generating...")
  with torch.no_grad():
      generated_output = base_model.generate(
              input_ids=tokens,
              generation_config=generation_config,
          )

  generated_output_tokens = generated_output[0][length:]
  output = tokenizer.decode(generated_output_tokens, skip_special_tokens=True)
  return output

In [ ]:
%%time
# same prompt as provided in https://crfm.stanford.edu/2023/03/13/alpaca.html
instruction = r'What is an alpaca? How is it different from a llama?'
'''
instruction = r'Write an e-mail to congratulate new Standford admits and mention that you are excited about meeting all of them in person.'
instruction = r'What is the capital of Tanzania?'
instruction = r'Write a well-thought out abstract for a machine learning paper that proves that 42 is the optimal seed for training neural networks.'
'''

output = use_OpenAlpaca(input=instruction)
print(f'[!] Generation results: {output}')

[!] Generation results: Alpacas are smaller relatives of llamas and camels. They have shorter, softer hair and smaller ears, and they typically measure 1.5 to 2 meters (4.9 to 6.6 feet) in length and weigh between 30 and 75 kilograms (66 to 165 pounds).
CPU times: user 17.4 s, sys: 338 ms, total: 17.7 s
Wall time: 21.2 s


# Test generating with a pipeline

In [ ]:
# LangChain stuff
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

Response:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

In [ ]:
%%time
question = "What is an alpaca? How is it different from a llama?"

print(llm_chain.run(question))

 Alpacas are smaller relatives of llamas and camels. They have shorter, softer hair and smaller ears, and they typically measure 1.5 to 2 meters (4.9 to 6.6 feet) in length and weigh between 30 and 75 kilograms (66 to 165 pounds).
CPU times: user 15.5 s, sys: 74.1 ms, total: 15.6 s
Wall time: 16.7 s


It's noteworthy that the HF pipeline doesn't generate as "good" an answer as the method.

# Test using LangChain Tools


## Create a LLM wrapper

References:
*   https://python.langchain.com/docs/modules/model_io/models/llms/how_to/custom_llm
*   https://betterprogramming.pub/creating-my-first-ai-agent-with-vicuna-and-langchain-376ed77160e3
*   Also see basically every tutorial on Agents in langchain

I believe that the LLM wrapper and using `initialze_agent()` is the highest level way of making an agent and passing an LLM chain to an LLMSingleActionAgent is used for deeper level customization?



In [ ]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

class LocalAlpaca(LLM):
    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return use_OpenAlpaca(prompt)

In [ ]:
wrapped_llm = LocalAlpaca()
print(wrapped_llm)

LocalAlpaca
Params: {}


In [ ]:
%%time
question = "What is an alpaca? How is it different from a llama?"
print(wrapped_llm(question))

Alpacas are smaller relatives of llamas and camels. They have shorter, softer hair and a smaller head than lamas or camels. Their fiber is also softer and higher quality than wool.
CPU times: user 10.9 s, sys: 24.1 ms, total: 10.9 s
Wall time: 11 s


In [ ]:
%%time
print(wrapped_llm("What is 17 times 10?"))

170
CPU times: user 1.06 s, sys: 3.15 ms, total: 1.07 s
Wall time: 1.06 s


## Test a Python Agent

In [ ]:
# https://python.langchain.com/en/latest/modules/agents/toolkits/examples/python.html
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

# If I want to use PythonREPL directly I think
# from langchain.utilities import PythonREPL

In [ ]:
agent_executor = create_python_agent(
    llm=wrapped_llm,
    tool=PythonREPLTool(),
    verbose=True,
    handle_parsing_errors=True,
)

In [ ]:
print(agent_executor.agent.llm_chain.prompt)

input_variables=['input', 'agent_scratchpad'] output_parser=None partial_variables={} template='You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n\n\nPython_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Python_REPL]\nAction Input: the input to the a

In [ ]:
print(agent_executor.agent.llm_chain.prompt.template)

You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.


Python_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Python_REPL]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times

In [ ]:
%%time
agent_executor.run("What is 17 times 10?")



> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1327 in time            │
│                                                                                                  │
│   1324 │   │   if mode=='eval':                                                                  │
│   1325 │   │   │   st = clock2()                                                                 │
│   1326 │   │   │   try:                                                                          │
│ ❱ 1327 │   │   │   │   out = eval(code, glob, local_ns)                                          │
│   1328 │   │   │   except:                                                                       │
│   1329 │   │   │   │   self.shell.showtraceback()                                                │
│   1330 │   │   │   │   return                                                                    │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:267 in run                      │
│                                                                                                  │
│   264 │   │   if args and not kwargs:                                                            │
│   265 │   │   │   if len(args) != 1:                                                             │
│   266 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 267 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[self.output_keys[0]]      │
│   268 │   │                                                                                      │
│   269 │   │   if kwargs and not args:                                                            │
│   270 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[self.output_keys[0]]       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:149 in __call__                 │
│                                                                                                  │
│   146 │   │   │   )                                                                              │
│   147 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   148 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 149 │   │   │   raise e                                                                        │
│   150 │   │   run_manager.on_chain_end(outputs)                                                  │
│   151 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   152 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:143 in __call__                 │
│                                                                                                  │
│   140 │   │   )                                                                                  │
│   141 │   │   try:                                                                               │
│   142 │   │   │   outputs = (                                                                    │
│ ❱ 143 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   144 │   │   │   │   if new_arg_supported                                                       │
│   145 │   │   │   │   else self._call(inputs)              

In [ ]:
# troubleshooting with the handle_parsing_errors parameter

agent_executor = create_python_agent(
    llm=wrapped_llm,
    tool=PythonREPLTool(),
    verbose=True,
    handle_parsing_errors="Check your output and make sure it conforms!", # I think this reruns the chain with this prompt?
    )

In [ ]:
%%time
agent_executor.run("What is 17 times 10?")



> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1327 in time            │
│                                                                                                  │
│   1324 │   │   if mode=='eval':                                                                  │
│   1325 │   │   │   st = clock2()                                                                 │
│   1326 │   │   │   try:                                                                          │
│ ❱ 1327 │   │   │   │   out = eval(code, glob, local_ns)                                          │
│   1328 │   │   │   except:                                                                       │
│   1329 │   │   │   │   self.shell.showtraceback()                                                │
│   1330 │   │   │   │   return                                                                    │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:267 in run                      │
│                                                                                                  │
│   264 │   │   if args and not kwargs:                                                            │
│   265 │   │   │   if len(args) != 1:                                                             │
│   266 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 267 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[self.output_keys[0]]      │
│   268 │   │                                                                                      │
│   269 │   │   if kwargs and not args:                                                            │
│   270 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[self.output_keys[0]]       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:149 in __call__                 │
│                                                                                                  │
│   146 │   │   │   )                                                                              │
│   147 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   148 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 149 │   │   │   raise e                                                                        │
│   150 │   │   run_manager.on_chain_end(outputs)                                                  │
│   151 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   152 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:143 in __call__                 │
│                                                                                                  │
│   140 │   │   )                                                                                  │
│   141 │   │   try:                                                                               │
│   142 │   │   │   outputs = (                                                                    │
│ ❱ 143 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   144 │   │   │   │   if new_arg_supported                                                       │
│   145 │   │   │   │   else self._call(inputs)              

## A second test using `initialize_agent` function

reference:
- https://python.langchain.com/docs/modules/agents/agent_types/self_ask_with_search

I could also try to add a wikipedia search here

### Setting up wikipedia

In [ ]:
!pip -q install wikipedia

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_shell.py:99 in system                      │
│                                                                                                  │
│    96 │   if pip_warn:                                                                           │
│    97 │     kwargs.update({'also_return_output': True})                                          │
│    98 │                                                                                          │
│ ❱  99 │   output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=pr   │
│   100 │                                                                                          │
│   101 │   if pip_warn:                                                                           │
│   102 │     _pip.print_previous_import_warning(output)                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_system_commands.py:453 in _system_compat   │
│                                                                                                  │
│   450   # We set a higher depth than the IPython system command since a shell object             │
│   451   # is expected to call this function, thus adding one level of nesting to the             │
│   452   # stack.                                                                                 │
│ ❱ 453   result = _run_command(                                                                   │
│   454 │     shell.var_expand(cmd, depth=2), clear_streamed_output=False                          │
│   455   )                                                                                        │
│   456   shell.user_ns['_exit_code'] = result.returncode                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_system_commands.py:167 in _run_command     │
│                                                                                                  │
│   164   """Calls the shell command, forwarding input received on the stdin_socket."""            │
│   165   locale_encoding = locale.getpreferredencoding()                                          │
│   166   if locale_encoding != _ENCODING:                                                         │
│ ❱ 167 │   raise NotImplementedError(                                                             │
│   168 │   │   'A UTF-8 locale is required. Got {}'.format(locale_encoding)                       │
│   169 │   )                                                                                      │
│   170                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
from langchain.utilities import WikipediaAPIWrapper

In [ ]:
wikipedia = WikipediaAPIWrapper()

In [ ]:
wikipedia.run('Langchain')

### Setting up python REPL

In [ ]:
from langchain.utilities import PythonREPL

In [ ]:
python_repl = PythonREPL()

In [ ]:
python_repl.run("print(17*2)")

### Set up tools

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

In [ ]:
tools = [
    Tool(
        name = "python repl",
        func=python_repl.run,
        description="useful for when you need to use python to answer a question. You should input python code"
    )
]

In [ ]:
wikipedia_tool = Tool(
    name='wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia"
)
tools.append(wikipedia_tool)

In [ ]:
# Using wrapped llm
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=wrapped_llm,
    verbose=True,
    max_iterations=3,
)

In [ ]:
zero_shot_agent.run("When was Barak Obama born?")

I continue to struggle with getting the model to output correctly...

ref: https://github.com/hwchase17/langchain/issues/1358